#Environment setup

Necessary libraries

In [ ]:
!pip install langchain langchain_community faiss-cpu

In [3]:
from google.colab import files
import zipfile
import os
import faiss
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.document_loaders.word_document import Docx2txtLoader
from langchain_community.document_loaders import DirectoryLoader

Global variables

In [4]:
db_path = "/content/data"
api_token = "your api token here"

#Data uploading

In [ ]:
uploaded = files.upload()

In [ ]:
!unzip /content/data.zip -d /content/data

#Body part

In [ ]:
# Load all .txt files from a directory
docs = []

for root, dirs, files in os.walk(db_path):
    for f in files:
        if f.endswith('.txt'):
            file_path = os.path.join(root, f)
            loader = TextLoader(file_path)
            docs.extend(loader.load())

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100, )
documents = text_splitter.split_documents(docs)

# Define the embedding model
model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

# Create the vector store
vector = FAISS.from_documents(documents, embeddings)

# Define a retriever interface
retriever = vector.as_retriever()

In [ ]:
# Define LLM
llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.3",
                          task="text-generation",
                          huggingfacehub_api_token=api_token,
                          max_new_tokens = 500,)
model = ChatHuggingFace(llm=llm)

# Define prompt template
prompt = ChatPromptTemplate.from_template("""Старайся отвечать вежливо и точно. Ответь на поставленный вопрос в соответствии с контекстом

<context>
{context}
</context>

Вопрос: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
query = '''
Как позвонить на Горячую линию Cекретариата Российского совета олимпиад школьников?
'''
response = retrieval_chain.invoke({"input": query})
print(response["answer"])